### Importar librerías

In [ ]:
import pandas as pd
import numpy as np

import ast

from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


### Cargar y preparar los datos

In [ ]:
# Cargamos el dataset
df = pd.read_csv('../datos/df_final.csv')
df.head()

### Preprocesamiento de los datos

In [30]:
# Separamos las variables de entrada y salida
columnas_entrada = ['vel alambre', 'flujo gas', 'peri voltaje', 'voltaje', 'corriente', 'ubicacion']

# Conjunto de entrada
X = df[columnas_entrada]
print(X.shape)

# Conjunto de salida
y = df['matriz']
print(y.dtypes)

(54, 6)
object


In [37]:
def clean_string(s):
    # Reemplazar múltiples puntos por comas
    s = re.sub(r'\.\.', ',', s)
    # Quitar cualquier punto al final de la cadena
    s = re.sub(r'\.$', '', s)
    # Reemplazar cualquier punto aislado por coma
    s = s.replace('.', ',')
    # Asegurarse de que los números están separados por comas
    s = re.sub(r'(\d),(\d)', r'\1,\2', s)
    return s

# Aplicar la función de limpieza a todas las entradas de y
y = y.apply(clean_string)

In [38]:
def parse_matrix_string(s):
    # Evaluar la cadena como una lista de listas
    return np.array([list(map(int, row.split(','))) for row in re.findall(r'\[(.*?)\]', s)])

y = y.apply(parse_matrix_string)

ValueError: invalid literal for int() with base 10: '[1'

In [34]:
# Conjunto de entrada
print('Conjunto de entrada')
print(X.head())

# Salto de linea
print('\n')

# Conjunto de salida
print('Conjunto de salida')
print("Forma de y:", y.shape)
print(y.shape)
print(y[0])
print(y.dtypes)
print(type(df['matriz'].iloc[0]))

Conjunto de entrada
   vel alambre  flujo gas  peri voltaje  voltaje  corriente  ubicacion
0          5.0        5.0           5.0      5.0        5.0          1
1          4.0        4.0           4.0      4.0        4.0          1
2          4.0        4.0           4.0      4.0        4.0          1
3          2.0        2.0           2.0      2.0        2.0          1
4          3.0        3.0           3.0      3.0        3.0          1


Conjunto de salida
Forma de y: (54,)
(54,)
[[1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
object
<class 'str'>


### Dividir los datos en conjuntos de entrenamiento y prueba

In [ ]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Imprimir las formas
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

### Crear el modelo

In [ ]:
# Número de características
num_features = X_train.shape[1]

funciones_activacion = ['relu', 'sigmoid', 'tanh']
activation_hidden_layer = funciones_activacion[2]  # Capas ocultas
activation_output_layer = funciones_activacion[1]  # Capa de salida

# Crear el modelo
model = Sequential([
    Dense(8, activation=activation_hidden_layer, input_shape=(num_features,)),
    Dense(16, activation=activation_hidden_layer),
    Dense(32, activation=activation_hidden_layer),
    Dense(y.shape[0], activation=activation_output_layer)
])

In [ ]:
print(y.shape[0])

In [ ]:
# Compilar el modelo

# Crear el optimizador con una tasa de aprendizaje ajustada
optimizer = Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

### Entrenar el modelo

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10000, verbose=2)

### Evaluar el modelo

In [ ]:
# Evaluar el modelo en el conjunto de datos de prueba
loss, mae = model.evaluate(X_train, y_train)
print(f'Loss (MSE): {loss},\n MAE: {mae}')

### Visualizar del entrenamiento

In [ ]:
# Obtener el historial de pérdida y métricas
loss = history.history['loss']
mae = history.history['mean_absolute_error']

In [ ]:
# Crear gráficos
plt.figure(figsize=(12, 5))

# Pérdida
plt.subplot(1, 2, 1)
plt.plot(loss, label='Training Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(mae, label='Training MAE')
plt.title('Mean Absolute Error (MAE)')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()

# Mostrar los gráficos
plt.tight_layout()  # Ajusta el espacio entre subplots
plt.show()

### Predicciones

In [ ]:
index = 2
X_ejemplo = X_test[index:index+1]

# Realizar la predicción
y_pred = model.predict(X_ejemplo)

umbral = 0.5

prediccion_binaria = (y_pred >= umbral).astype(int)
valor_real = y_test[index]

# Imprimir la predicción y el valor real
print("Predicción:", prediccion_binaria)
print("Valor real:", valor_real)

# Si el valor de salida es una matriz 2D, imprimir las formas para confirmar
print("Forma de la predicción:", prediccion_binaria.shape)
print("Forma del valor real:", valor_real.shape)

In [ ]:
table_data = np.vstack([valor_real, prediccion_binaria.flatten()]).T

col_labels = ['Valor Real', 'Predicción Binaria']

fig, ax = plt.subplots(figsize=(10, len(valor_real) * 0.5))  # Ajustar el tamaño de la figura según la cantidad de datos
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=table_data, colLabels=col_labels, cellLoc='center', loc='center')

# Ajustar el tamaño de la fuente
table.auto_set_font_size(False)
table.set_fontsize(12)

# Ajustar el tamaño de las columnas
table.auto_set_column_width(col_labels)

# Agregar bordes a las celdas
for key, cell in table._cells.items():
    cell.set_edgecolor('black')
    cell.set_linewidth(1.5)

# Agregar color de fondo a los encabezados
for (i, label) in enumerate(col_labels):
    cell = table[(0, i)]
    cell.set_text_props(weight='bold', color='white')
    cell.set_facecolor('#40466e')

# Ajustar el espaciado de las celdas
for key, cell in table._cells.items():
    cell.set_fontsize(12)
    cell.set_edgecolor('black')
    cell.set_linewidth(1.5)

# Mostrar la tabla
plt.show()